In [39]:
# make the screen bigger!
from IPython.display import display, HTML
display(HTML(data=""" <style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 85%; }
    div#maintoolbar-container { width: 99%; } </style> """))


#### import modules
import numpy as np
import arcpy
import os
import sys
from arcpy.sa import *
import pandas as pd
from arcpy import env
import shutil
import numpy.ma as ma
import netCDF4 as nc
import subprocess
import re
import datetime
import matplotlib 
import matplotlib.pyplot as plt
import scipy
from scipy.stats import linregress
import scipy.optimize as opt

# Tired of pink boxes?
pd.options.mode.chained_assignment = None  # default='warn'

# set properties
arcpy.env.overwriteOutput = True # make sure overwrite files is on
# projection definition 
sr_project = arcpy.SpatialReference(32702)   # Project dataset into WGS84

input_folder = os.path.join("..", "input")  


GIS_FOLDER = os.path.join('..', '..', 'Raw_GIS_Data')
#STD_INPUT_FOLDER = os.path.join('..', '..', 'Std_input')

# path to the grid bound
Grid_shp = os.path.join(GIS_FOLDER, 'grid_bound.shp')


In [21]:
def mod_ctrl_file_cellSize(Control_File_Name):
    
    # modify the control file gridsize for the given run   (note this uses dimensions from the rainfall adjustment grid in april)
    with open(os.path.join(input_folder, 'RF_adj_grids', 'rfadj_apr.asc'), 'r') as dims_file:   # open an ASC file and get the dimenstions out of it 
        dimsfile1 = dims_file.read().splitlines(True)
        x_dim = float(re.findall('\d+', dimsfile1[0])[-1])    
        y_dim = float(re.findall('\d+', dimsfile1[1])[-1])
        cel_size = float(re.findall('\d+', dimsfile1[4])[-1])

    with open(os.path.join('.', Control_File_Name), 'r') as fin:   # open file 
        data = fin.read().splitlines(True)
    with open(os.path.join('.', Control_File_Name), 'w') as fout:     # delete first line
        fout.writelines(data[1:])
    new_first = 'GRID {} {} 515000. 8409000. {} '.format(x_dim, y_dim, cel_size)  # new first line 
    with open(os.path.join('.', Control_File_Name), 'r+') as file:                # add in new first line and save file  
        file_data = file.read()
        file. seek(0, 0)
        file. write(new_first + '\n' + file_data)
        
        return x_dim, y_dim, cel_size

# Define Run model function


Changes needed to the function
- Specify individual output folders (no making them, need to make them in the cells below 
- Specify individual control files (all Ctrl files should hang out in the run folder with the swb2.exe
                            


### Refactored 6-24-2020
                
    

In [22]:
def run_model(Control_File_Name, output_folder, x_dim, y_dim, Direct_infiltration_input_folder):
    print(Control_File_Name)
    
    # this is all to re-copy over all the input files from the direct infimtration from OSDS and waterlines in order to add in MFR
    Dir_infil_imp = os.path.join("..", "input", Direct_infiltration_input_folder)

    # Copy over the original direct infiltration grid from the input ata generation script
    Temp_MFR_folder = os.path.join("..", "Temp_MFR")
    shutil.copy2(os.path.join(".." , 'input', Dir_infil_imp, "total_inlf_in.asc"), Temp_MFR_folder) 
    shutil.copy2(os.path.join(".." , 'input', Dir_infil_imp, "WLOSDrast.aux.xml"), Temp_MFR_folder) 
    if os.path.exists(os.path.join(Temp_MFR_folder, "wlosdrast")):   # gotta do this to avoid an folder already exists error
        shutil.rmtree(os.path.join(Temp_MFR_folder, "wlosdrast"))
    shutil.copytree(os.path.join(".." , 'input', Dir_infil_imp, "wlosdrast"),  os.path.join(Temp_MFR_folder, "wlosdrast"))

    # RUN Da MODEL (with no MFR) 
    print("Run model 1, no MFR")
    os.chdir(os.path.join("..", "run"))
    # Executable and control file copies
    shutil.copy2(os.path.join("." , 'swb2.exe') ,output_folder) 
    shutil.copy2(os.path.join("." , Control_File_Name) ,output_folder) 

    os.chdir(output_folder)
    subprocess.call('swb2.exe {}'.format(Control_File_Name), shell=True)
    os.chdir(os.path.join("..", "run"))

    ### Post process da files
    outspace = os.path.join(output_folder, 'post_prcessed_no_MFR')
    if not os.path.exists(outspace):
        os.makedirs(outspace)

    # Parameters
    Desired_files = ['actual_et',  'direct_net_infiltation', 'direct_soil_moisture',
                 'interception', 'net_infiltration', 'rainfall', 'runoff'] # 'delta_soil_storage',  'irrigation', 
    XLLCORNER =      515000.000
    YLLCORNER =      8409000.000
    CELLSIZE  =      cel_size

    # functions
    def create_file_reference( component_name ):
        '''
        This is a simple convenience function that will form a path and filename to a
        given water budget component
        '''
        # specify the prefix, path to SWB2 output, timeframe, and resolution
        #output_path = os.path.join(os.getcwd(), "output")
        #prefix      = '\\'
        
    # pull start and end dates from control file
        with open(os.path.join('.', Control_File_Name), 'r') as fin:   # open file 
            date_find = fin.read().splitlines(True)
        start_year = datetime.datetime.strptime(date_find[-2].split(" ")[1], "%m/%d/%Y").strftime("%Y-%m-%d")
        end_year =  datetime.datetime.strptime(date_find[-1].split(" ")[1], "%m/%d/%Y").strftime("%Y-%m-%d")

        ncol        = str(int(x_dim))
        nrow        = str(int(y_dim))
        return(  component_name + '__' + start_year + '_' 
              + end_year + '__' + nrow + '_by_' + ncol + '.nc' )

    # some other functions to post process stuff

    def ncdump(nc_fid):
        '''ncdump outputs dimensions, variables and their attribute information of netCDF4 files'''
        nc_attrs = nc_fid.ncattrs()
        nc_dims = [dim for dim in nc_fid.dimensions]  
        nc_vars = [var for var in nc_fid.variables] 
        return nc_attrs, nc_dims, nc_vars

    def writeArrayToArcGrid(arr,filename,xll,yll,cellsize,no_data_val):
        """ this takes a 2d numpy array and turns it into an .asc file """
        arr                = np.copy(arr)
        arr[np.isnan(arr)] = no_data_val
        headerstring       = bytes('NCOLS %d\nNROWS %d\nXLLCENTER %f\nYLLCENTER %f\nCELLSIZE %f\nNODATA_value %f\n' % 
            (arr.shape[1], arr.shape[0], xll, yll, cellsize, no_data_val), 'UTF-8')

        with open(filename,'wb') as fout:
            fout.write(headerstring)
            np.savetxt(fout,arr,'%5.2f')


    # post process the whole model domain
    var = []; tot = []; nclist = []

    # Step 1: make list of files that you wish to process 
    for i in Desired_files:
        Da_file = create_file_reference(i)
        nclist.append(os.path.join(output_folder, Da_file))

    # Step 2 average the daily dimension (len(t) is # of days in the run) to annual 
    for i, f in enumerate(nclist):
        nc_data = nc.Dataset(nclist[i])
        nc_attrs, nc_dims, nc_vars = ncdump(nc_data)
        nc_var = nc_vars[3]
        t = nc_data.variables['time'][:]
        y = nc_data.variables['y'][:]
        x = nc_data.variables['x'][:]
        nt = len(t)
        nrow = len(y)
        ncol = len(x)
        rd = np.zeros((nrow, ncol))  # create 0 array of the proper shape
        for day in range(nt):
            r_temp = nc_data.variables[str(nc_var)][day, :, :]
            r_filled = np.ma.filled(r_temp, fill_value=0)    # fills in missing values with 0s (i think) 
            rd = rd+r_filled                                 # sequentially add each day's value in each cell to the empty frame  
        r = rd/nt*365 # to create a one year average from all the years in model.  if want to add leap years add 0.25 

        # step 3: write each yearly average array to a .asc file
        keyname = Desired_files[i] 
        writeArrayToArcGrid(r, os.path.join(outspace, "{}_annual.asc".format(keyname)), XLLCORNER, YLLCORNER, CELLSIZE, -999)

        # Step 4: calculate total amounts of water in cubic meters per day and create statistics dataframe
        m3pd = ((cel_size**2)*r.sum()*.0254)/365 
        
        var.append(keyname) ; tot.append(m3pd)     # make lists to populate pandas dataframe

        nc_data.close()          # make sure to close the nc file so it doesnt stay open

    stat_frame = pd.DataFrame({'Variable' : var, 'total_[m3pd]': tot})    #in case you want the max and min#, "Max_[in]": mx, "Min_[in]":mn})
    stat_frame["total_[MGD]"] = stat_frame["total_[m3pd]"]/3785.41178       # put things in MGD if interested, 3785.41178 is number of gal in m3      
    Precip = list(stat_frame[stat_frame['Variable'] == 'rainfall']['total_[m3pd]'])[0]   # define the amount of calculated Precip
    Dir_net_inf = list(stat_frame[stat_frame['Variable'] == 'direct_net_infiltation']['total_[m3pd]'])[0]   # define the amount of calculated infiltration
    WB_ins = Precip + Dir_net_inf
    stat_frame['pct_of_pcip'] = stat_frame["total_[m3pd]"]/WB_ins
    stat_frame.to_csv(os.path.join(outspace, "stats_run7_{}m_cells.csv".format(cel_size)))

    # how does the model balance? 
    print("WATER BALANCE ratio: outs over ins water budget balanece =  {} % ".format(stat_frame['pct_of_pcip'].sum()-1))   # check water balance


    # calculate statistics for individual watersheds
    # note, for some reason will not overwrite csvs need to clear them out or recode to make this issue not an issue
    #create workspace
    outspace_table = os.path.join(output_folder, 'post_prcessed_no_MFR', "tables")
    if not os.path.exists(outspace_table):
        os.makedirs(outspace_table)
    sheds = (os.path.join(GIS_FOLDER, 'Watersheds\\Runoff_zones_sheds_WGS2S_clip.shp'))

    # process each raster layer into a table
    for i in (os.listdir(outspace)):
        if i.endswith('.asc'):
            outZSaT = ZonalStatisticsAsTable(sheds, "SHED_NAME", os.path.join(outspace, i), os.path.join(outspace_table, "temptab.dbf"))  # in arc format
            arcpy.TableToTable_conversion(outZSaT, outspace_table, "Table_{}_1.csv".format(i))                                            # take table out of stupid arc format and put into csv format 

    # this block takes each of the csvs, reads them and calculates water volumnes (m3/d) for each watershed
    templist = []
    for c in (os.listdir(os.path.join(outspace, "tables"))):
        if c.endswith('.csv'):
            data = pd.read_csv(os.path.join(outspace, "tables", c))
            keyname = c.split("Table_")[1].split("_annual")[0]                   # parameter being worked on
            data[keyname] = (data['MEAN']*.0254/365) * data['AREA'] 
            temp_frame = data[["SHED_NAME", keyname]]
            templist.append(temp_frame)

    summarry_frame1 = data[['SHED_NAME']]                                        # this is just sticking them all together into one dataframe
    for i in templist:
        summarry_frame1 = summarry_frame1.merge(i, on ='SHED_NAME', how='outer')


    # that was in actual volumns, now to convert each component into a fraction of the rainfall...
    templist2 = []
    summarry_frame2 = data[['SHED_NAME']]
    for i in summarry_frame1.columns[1:]:
        temp_frame = data[['SHED_NAME']] ; temp_frame[i.split("-")[0]] = summarry_frame1[i]/summarry_frame1['rainfall']
        templist2.append(temp_frame)

    summarry_frame3 = data[['SHED_NAME']]
    for i in templist2:
        summarry_frame3 = summarry_frame3.merge(i, on ='SHED_NAME', how='outer')

    summarry_frame_4000 = summarry_frame1.set_index('SHED_NAME')
    summarry_frame_4 = summarry_frame_4000.select_dtypes(exclude=['object'])*264.172/1000000   # convert to million gallons per day

    summarry_frame3.to_csv(os.path.join(outspace, "watershed_summary_stats_percentages.csv"))
    summarry_frame1.to_csv(os.path.join(outspace, "watershed_summary_stats_volume_m3pd.csv"))
    summarry_frame_4.to_csv(os.path.join(outspace, "watershed_summary_stats_volumes_MGD.csv"))


    ### MFR calculations      
    outspace = os.path.join(output_folder, 'post_prcessed_no_MFR')
    if not os.path.exists(outspace):
        os.makedirs(outspace)

    # caclulate how much runoff to dump into the MFR area
    outspace_table = os.path.join(output_folder, 'MFR_calcs', "tables")
    if not os.path.exists(outspace_table):
        os.makedirs(outspace_table)

    Contributing_area_leo = (os.path.join(GIS_FOLDER, 'MFR\\Contributing_MRF_Areas_leone.shp'))
    Contributing_area_taf = (os.path.join(GIS_FOLDER, 'MFR\\Contributing_MRF_Areas_tafuna.shp'))

    outZSaT_leo = ZonalStatisticsAsTable(Contributing_area_leo, "SHED_NAME", os.path.join(outspace, "runoff_annual.asc"), os.path.join(outspace_table, "temptab_leo.dbf"))  # in arc format
    arcpy.TableToTable_conversion(outZSaT_leo, outspace_table, "runoff_MFR_leo.csv")                                            # take table out of stupid arc format and put into csv format 
    outZSaT_leo = ZonalStatisticsAsTable(Contributing_area_taf, "SHED_NAME", os.path.join(outspace, "runoff_annual.asc"), os.path.join(outspace_table, "temptab_taf.dbf"))  # in arc format
    arcpy.TableToTable_conversion(outZSaT_leo, outspace_table, "runoff_MFR_taf.csv") 

    data_leo = pd.read_csv(os.path.join(outspace_table, "runoff_MFR_leo.csv"))
    data_taf = pd.read_csv(os.path.join(outspace_table, "runoff_MFR_taf.csv"))

    data_leo["AreaRunoff_m3pd"] = (data_leo['MEAN']*.0254/365) * data_leo['AREA']    # this is how much runoff is in each MFR contributionzone
    data_taf["AreaRunoff_m3pd"] = (data_taf['MEAN']*.0254/365) * data_taf['AREA']    # this is how much runoff is in each MFR contributionzone
    tot_MFR_leo = sum(data_leo['AreaRunoff_m3pd'])
    tot_MFR_taf = sum(data_taf['AreaRunoff_m3pd'])

    # calculate the MFR area and prepare new input files
    workspace = os.path.join(Temp_MFR_folder,  'MFR')
    if not os.path.exists(workspace):
        os.makedirs(workspace)

    arcpy.Project_management(os.path.join(GIS_FOLDER, 'MFR\\MFR_infiltration_area_leone.shp'),  os.path.join(workspace, 'MFR_infiltration_area_leone_projected.shp'), sr_project) 
    arcpy.AddField_management(os.path.join(workspace, 'MFR_infiltration_area_leone_projected.shp'), "MFR_inch", "DOUBLE")    # add Active cell unit field
    arcpy.AddGeometryAttributes_management(os.path.join(workspace, 'MFR_infiltration_area_leone_projected.shp'), "AREA")
    Total_MFR_area_leo = 0                                                                                                        # stupid block just to calculate the total area
    with arcpy.da.SearchCursor(os.path.join(workspace, 'MFR_infiltration_area_leone_projected.shp'), "POLY_AREA") as cursor:
        for row in cursor:
            Total_MFR_area_leo = Total_MFR_area_leo + row[0]

    arcpy.Project_management(os.path.join(GIS_FOLDER, 'MFR\\MFR_infiltration_area_tafuna.shp'),  os.path.join(workspace, 'MFR_infiltration_area_tafuna_projected.shp'), sr_project) 
    arcpy.AddField_management(os.path.join(workspace, 'MFR_infiltration_area_tafuna_projected.shp'), "MFR_inch", "DOUBLE")    # add Active cell unit field
    arcpy.AddGeometryAttributes_management(os.path.join(workspace, 'MFR_infiltration_area_tafuna_projected.shp'), "AREA")
    Total_MFR_area_taf = 0                                                                                                        # stupid block just to calculate the total area
    with arcpy.da.SearchCursor(os.path.join(workspace, 'MFR_infiltration_area_tafuna_projected.shp'), "POLY_AREA") as cursor:
        for row in cursor:
            Total_MFR_area_taf = Total_MFR_area_taf + row[0]

    Inches_of_MFR_across_leo = (tot_MFR_leo/Total_MFR_area_leo/0.0254) * 0.75   # note this 75% number if directly from Izuka 2007
    Inches_of_MFR_across_taf = (tot_MFR_taf/Total_MFR_area_taf/0.0254) * 0.75   # note this 75% number if directly from Izuka 2007

    arcpy.CalculateField_management(os.path.join(workspace, 'MFR_infiltration_area_leone_projected.shp'), "MFR_inch", "!MFR_inch! + {}".format(Inches_of_MFR_across_leo), "PYTHON3") # calculate the appropriate amount of infitration in inches spread over all MFR zone
    arcpy.CalculateField_management(os.path.join(workspace, 'MFR_infiltration_area_tafuna_projected.shp'), "MFR_inch", "!MFR_inch! + {}".format(Inches_of_MFR_across_taf), "PYTHON3") # calculate the appropriate amount of infitration in inches spread over all MFR zone

    arcpy.Erase_analysis(Grid_shp, os.path.join(workspace, 'MFR_infiltration_area_leone_projected.shp'),  os.path.join(workspace, 'MFR_infiltration_area_leone_bound.shp'))
    arcpy.Erase_analysis(Grid_shp, os.path.join(workspace, 'MFR_infiltration_area_tafuna_projected.shp'),  os.path.join(workspace, 'MFR_infiltration_area_tafuna_bound.shp'))

    arcpy.Merge_management([os.path.join(workspace, 'MFR_infiltration_area_leone_bound.shp'), os.path.join(workspace, 'MFR_infiltration_area_leone_projected.shp')], os.path.join(workspace, 'MFR_infiltration_area_leone_ready.shp'))
    arcpy.Merge_management([os.path.join(workspace, 'MFR_infiltration_area_tafuna_bound.shp'), os.path.join(workspace, 'MFR_infiltration_area_tafuna_projected.shp')], os.path.join(workspace, 'MFR_infiltration_area_tafuna_ready.shp'))

    arcpy.PolygonToRaster_conversion(os.path.join(workspace, 'MFR_infiltration_area_leone_ready.shp'), "MFR_inch", os.path.join(workspace, "MFR_Rast_L"), cell_assignment="MAXIMUM_AREA",  cellsize=cel_size)
    arcpy.PolygonToRaster_conversion(os.path.join(workspace, 'MFR_infiltration_area_tafuna_ready.shp'), "MFR_inch", os.path.join(workspace, "MFR_Rast_T"), cell_assignment="MAXIMUM_AREA",  cellsize=cel_size)

    arcpy.RasterToASCII_conversion(os.path.join(workspace, "MFR_Rast_L"), os.path.join(workspace, "MFR_Rast_L.asc"))
    arcpy.RasterToASCII_conversion(os.path.join(workspace, "MFR_Rast_T"), os.path.join(workspace, "MFR_Rast_T.asc"))

    arcpy.Delete_management(os.path.join(workspace, 'MFR_infiltration_area_leone_projected.shp'))
    arcpy.Delete_management(os.path.join(workspace, 'MFR_infiltration_area_tafuna_projected.shp'))
    arcpy.Delete_management(os.path.join(workspace, 'MFR_infiltration_area_leone_bound.shp'))
    arcpy.Delete_management(os.path.join(workspace, 'MFR_infiltration_area_tafuna_bound.shp'))
    arcpy.Delete_management(os.path.join(workspace, 'MFR_infiltration_area_leone_ready.shp'))
    arcpy.Delete_management(os.path.join(workspace, 'MFR_infiltration_area_tafuna_ready.shp'))
    arcpy.Delete_management(os.path.join(workspace, 'MFR_Rast_L'))
    arcpy.Delete_management(os.path.join(workspace, 'MFR_Rast_T'))
    

    # now combine the MFR raster into the other direct infiltration rasters
    arcpy.Plus_3d(os.path.join(workspace, "MFR_Rast_L.asc"), os.path.join(Temp_MFR_folder, "WLOSDrast"), os.path.join(Temp_MFR_folder, "temprast2"))
    arcpy.Plus_3d(os.path.join(workspace, "MFR_Rast_T.asc"), os.path.join(Temp_MFR_folder, "temprast2"), os.path.join(Temp_MFR_folder, "temprast3"))
    arcpy.RasterToASCII_conversion(os.path.join(Temp_MFR_folder, "temprast3"), os.path.join(Temp_MFR_folder, "Total_inlf_in.asc"))       ########################################HERERERERR!!!!!!!!!!!!!!!!!!!!

    #print('MFR leo in MGD is {}'.format(tot_MFR_leo*264.172/1000000))
    #print('MFR taf in MGD is {}'.format(tot_MFR_taf*264.172/1000000))
    print('MFR total in MGD is {}'.format((tot_MFR_leo+tot_MFR_taf)*264.172/1000000))
    print("Caclculating MFR")
    
    # Run da Model again, this time including the MFR
    print("Run model 2, with MFR")
    
    # Executable and control file copies
    shutil.copy2(os.path.join("." , 'swb2.exe') ,os.path.join(output_folder)) 
    shutil.copy2(os.path.join("." , Control_File_Name) ,os.path.join(output_folder)) 

    os.chdir(os.path.join(output_folder))
    subprocess.call('swb2.exe {}'.format(Control_File_Name), shell=True)
    os.chdir(os.path.join("..", "run"))

    # Post process the files again, this time with the MFR added 
    outspace = os.path.join(output_folder, 'post_prcessed_with_MFR')
    if not os.path.exists(outspace):
        os.makedirs(outspace)

    # post process the whole model domain      
    var = []; tot = []; nclist = []

    # Step 1: make list of files that you wish to process 
    for i in Desired_files:
        Da_file = create_file_reference(i)
        nclist.append(os.path.join(output_folder, Da_file))

    # Step 2 average the daily dimension (len(t) is # of days in the run) to get a frame of annual stats
    for i, f in enumerate(nclist):
        nc_data = nc.Dataset(nclist[i])
        nc_attrs, nc_dims, nc_vars = ncdump(nc_data)
        nc_var = nc_vars[3]
        t = nc_data.variables['time'][:]
        y = nc_data.variables['y'][:]
        x = nc_data.variables['x'][:]
        nt = len(t)
        nrow = len(y)
        ncol = len(x)
        rd = np.zeros((nrow, ncol))  # create 0 array of the proper shape
        for day in range(nt):
            r_temp = nc_data.variables[str(nc_var)][day, :, :]
            r_filled = np.ma.filled(r_temp, fill_value=0)    # fills in missing values with 0s (i think) 
            rd = rd+r_filled                                 # sequentially add each day's value in each cell to the empty frame  
        r = rd/nt*365 # to create a one year average from all the years in model.  if want to add leap years add 0.25 

        # step 3: write each yearly average array to a .asc file
        keyname = Desired_files[i] 
        writeArrayToArcGrid(r, os.path.join(outspace, "{}_annual.asc".format(keyname)), XLLCORNER, YLLCORNER, CELLSIZE, -999)

        # Step 4: calculate total amounts of water in cubic meters per day and create statistics dataframe
        m3pd = ((cel_size**2)*r.sum()*.0254)/365 
        # print("{} total  {} [m3/d]".format(keyname, '%.1f' % m3pd))
        var.append(keyname) ; tot.append(m3pd)     # make lists to populate pandas dataframe

        nc_data.close()          # make sure to close the nc file so it doesnt stay open

    stat_frame = pd.DataFrame({'Variable' : var, 'total_[m3pd]': tot})    #in case you want the max and min#, "Max_[in]": mx, "Min_[in]":mn})
    stat_frame["total_[MGD]"] = stat_frame["total_[m3pd]"]/3785.41178       # put things in MGD if interested, 3785.41178 is number of gal in m3  
    stat_frame["total_[Mld]"] =stat_frame["total_[MGD]"]*3.78541            # Convert to mld
    Precip = list(stat_frame[stat_frame['Variable'] == 'rainfall']['total_[m3pd]'])[0]   # define the amount of calculated Precip
    Dir_net_inf = list(stat_frame[stat_frame['Variable'] == 'direct_net_infiltation']['total_[m3pd]'])[0]   # define the amount of calculated infiltration
    WB_ins = Precip + Dir_net_inf
    stat_frame['pct_of_pcip'] = stat_frame["total_[m3pd]"]/WB_ins
    stat_frame.to_csv(os.path.join(outspace, "IslandWide_Annual_Summary_Stats.csv"))

    # how does the model balance? 
    print("WATER BALANCE ratio: outs over ins water budget balanece =  {} % ".format(stat_frame['pct_of_pcip'].sum()-1))   # check water balance
  
    
    # Step 2.2 average the daily dimension over months to get a frame of annual stats
    # Calculate MONTHLY island wide statistics 
    outspace_mon = os.path.join(output_folder, 'post_prcessed_with_MFR', "Monthly_summaries")
    if not os.path.exists(outspace_mon):
        os.makedirs(outspace_mon)

    results_frame = pd.DataFrame()

    # Read in the daily dimension (len(t) is # of days in the run) 
    for i, f in enumerate(nclist):                      # loop over each water budget component
        nc_data = nc.Dataset(nclist[i])                 # read in netcdef file 
        nc_attrs, nc_dims, nc_vars = ncdump(nc_data)    # grab dimension/variable headers
        nc_var = nc_vars[3]                             # the variabel of interest
        t = nc_data.variables['time'][:]
        y = nc_data.variables['y'][:]
        x = nc_data.variables['x'][:]
        nrow = len(y)
        ncol = len(x)

        # day numbers from the 1 year netcdf dataset that correspond to each month.  Will probably go bad in model is run longer
        Juli_months = {"JAN":t[0:31],  "FEB":t[31:59],  "MAR":t[59:90],  "APR":t[90:120 ],  "MAY":t[120:151 ],
                        "JUN":t[151:181 ],  "JUL":t[181:212 ],  "AUG":t[212:243 ],  
                                    "SEP":t[243:273 ],  "OCT":t[273:304 ],  "NOV":t[304:334 ],  "DEC":t[334:365]}

        for month in Juli_months.keys():

            dest_array = np.zeros((nrow, ncol))  # create 0 array of the proper shape
            nt = len(Juli_months[month])         # length of the month

            for day in Juli_months[month]:
                arr_temp = nc_data.variables[str(nc_var)][day, :, :]
                arr_filled = np.ma.filled(arr_temp, fill_value=0)    # fills in missing values with 0s (i think) 
                dest_array = dest_array+arr_filled                                 # sequentially add each day's value in each cell to the empty frame  
            fin_array = dest_array/nt*nt # to create the daily average value from all the years in model.  if want to add leap years add 0.25 

            # step 3: write each yearly average array to a .asc file 
            writeArrayToArcGrid(fin_array, os.path.join(outspace_mon, "{}_monthly_{}.asc".format(nc_var, month)), XLLCORNER, YLLCORNER, CELLSIZE, -999)

            # Step 4: calculate total amounts of water in cubic meters per day and create statistics dataframe
            m3pd = ((cel_size**2)*fin_array.sum()*.0254)/nt 
            #print("{}_{} total  {} [m3/d]".format(month, nc_var, '%.1f' % m3pd))

            results_frame.loc[month, nc_var] = m3pd
            results_frame.to_csv(os.path.join(outspace, "Monthly_summary_m3pd.csv"))

            results_frame_MGD = results_frame*(264.172/1000000)
            results_frame_MGD.to_csv(os.path.join(outspace, "Monthly_summary_MGD.csv"))

            results_frame_Mld = results_frame_MGD*3.78541 
            results_frame_Mld.to_csv(os.path.join(outspace, "Monthly_summary_Mld.csv"))

        nc_data.close()          # make sure to close the nc file so it doesnt stay open   
    

    # calculate statistics for individual watersheds
    # note, for some reason will not overwrite csvs need to clear them out or recode to make this issue not an issue
    #create workspace
    outspace_table = os.path.join(output_folder, 'post_prcessed_with_MFR', "tables")
    if not os.path.exists(outspace_table):
        os.makedirs(outspace_table)
    sheds = (os.path.join(GIS_FOLDER, 'Watersheds\\Runoff_zones_sheds_WGS2S_clip.shp'))

    # process each raster layer into a table
    for i in (os.listdir(outspace)):
        if i.endswith('.asc'):
            outZSaT = ZonalStatisticsAsTable(sheds, "SHED_NAME", os.path.join(outspace, i), os.path.join(outspace_table, "temptab.dbf"))  # in arc format
            arcpy.TableToTable_conversion(outZSaT, outspace_table, "Table_{}_1.csv".format(i))                                            # take table out of stupid arc format and put into csv format 

    # this block takes each of the csvs, reads them and calculates water volumnes (m3/d) for each watershed
    templist = []
    for c in (os.listdir(os.path.join(outspace, "tables"))):
        if c.endswith('.csv'):
            data = pd.read_csv(os.path.join(outspace, "tables", c))
            keyname = c.split("Table_")[1].split("_annual")[0]                   # parameter being worked on
            data[keyname] = (data['MEAN']*.0254/365) * data['AREA'] 
            temp_frame = data[["SHED_NAME", keyname]]
            templist.append(temp_frame)

    summarry_frame1 = data[['SHED_NAME']]                                        # this is just sticking them all together into one dataframe
    for i in templist:
        summarry_frame1 = summarry_frame1.merge(i, on ='SHED_NAME', how='outer')


    # that was in actual volumns, now to convert each component into a fraction of the rainfall...
    templist2 = []
    for i in summarry_frame1.columns[1:]:
        temp_frame = data[['SHED_NAME']] ; temp_frame[i.split("-")[0]] = summarry_frame1[i]/summarry_frame1['rainfall']
        templist2.append(temp_frame)

    summarry_frame3 = data[['SHED_NAME']]
    for i in templist2:
        summarry_frame3 = summarry_frame3.merge(i, on ='SHED_NAME', how='outer')

    # other units here
    summarry_frame_4000 = summarry_frame1.set_index('SHED_NAME')
    summarry_frame_4 = summarry_frame_4000.select_dtypes(exclude=['object'])/3785.41178    # convert to million gallons per day
    
    summarry_frame_5 = summarry_frame_4.select_dtypes(exclude=['object'])*3.78541              # convert to million liters per day 

    summarry_frame1.to_csv(os.path.join(outspace, "watershed_summary_stats_volume_m3pd.csv"))
    summarry_frame3.to_csv(os.path.join(outspace, "watershed_summary_stats_percentages.csv"))
    summarry_frame_4.to_csv(os.path.join(outspace, "watershed_summary_stats_volumes_MGD.csv"))
    summarry_frame_5.to_csv(os.path.join(outspace, "watershed_summary_stats_volumes_Mld.csv"))
    
    return summarry_frame1

# Run all the Scenarios
Each scenario has it's own cell

In [23]:
# BASE CASE: PRESENT DAY, HIGH RESOLUTION

Out_fold = os.path.join("..", "1_Original_run_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)
    
Control_File_Name = '1_Tutuila_cal_controlFile.ctl'
    
# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)

# RUN the model
Summary_frame_1 = run_model(Control_File_Name, Out_fold, x_dim, y_dim, Direct_infiltration_input_folder = "Direct_infiltration")


# delete the NetCDF files to clear up space (the data is already post processed into .asc's)
files_in_directory = os.listdir(Out_fold)
nc_files = [file for file in files_in_directory if file.endswith(".nc")]
for file in nc_files:
    path_to_file = os.path.join(Out_fold, file)
    os.remove(path_to_file)

1_Tutuila_cal_controlFile.ctl
Run model 1, no MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0036984073525663 % 
MFR total in MGD is 6.837100648803507
Caclculating MFR
Run model 2, with MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0036519123027903 % 


In [24]:
# Scenario 1  PRESENT DAY just for comparison to base case  **

""" CHANGES: 
LAND_USE ARC_GRID ../input/Land_use_Soils_data/lu_grid.asc   to
LAND_USE ARC_GRID ../input/Land_use_Soils_data_Scen1/lu_grid.asc

Be sure to use: 
Direct_infiltration_input_folder = "Direct_infiltration_Scen_1"  """


Out_fold = os.path.join("..", "2_scn1_noclimate_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)
    
Control_File_Name = '2_scn1_noclimate_controlFile.ctl'
    
# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)

# RUN the model
Summary_frame_1 = run_model(Control_File_Name, Out_fold, x_dim, y_dim, Direct_infiltration_input_folder = "Direct_infiltration_Scen_1")

# delete the NetCDF files to clear up space (the data is already post processed into .asc's)
files_in_directory = os.listdir(Out_fold)
nc_files = [file for file in files_in_directory if file.endswith(".nc")]
for file in nc_files:
    path_to_file = os.path.join(Out_fold, file)
    os.remove(path_to_file)

2_scn1_noclimate_controlFile.ctl
Run model 1, no MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.010697510716482 % 
MFR total in MGD is 7.059810489229983
Caclculating MFR
Run model 2, with MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0105594139372265 % 


In [25]:
# Scenario 2  PRESENT DAY just for comparison to base case  **

""" CHANGES: 
LAND_USE ARC_GRID ../input/Land_use_Soils_data/lu_grid.asc   to
LAND_USE ARC_GRID ../input/Land_use_Soils_data_Scen2/lu_grid.asc

Be sure to use: 
Direct_infiltration_input_folder = "Direct_infiltration_Scen_2"   """


Out_fold = os.path.join("..", "3_scn2_noclimate_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)
    
Control_File_Name = '3_scn2_noclimate_controlFile.ctl'
    
# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)

# RUN the model
Summary_frame_1 = run_model(Control_File_Name, Out_fold, x_dim, y_dim, Direct_infiltration_input_folder = "Direct_infiltration_Scen_2")

# delete the NetCDF files to clear up space (the data is already post processed into .asc's)
files_in_directory = os.listdir(Out_fold)
nc_files = [file for file in files_in_directory if file.endswith(".nc")]
for file in nc_files:
    path_to_file = os.path.join(Out_fold, file)
    os.remove(path_to_file)

3_scn2_noclimate_controlFile.ctl
Run model 1, no MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.000766761865966 % 
MFR total in MGD is 6.805638248914377
Caclculating MFR
Run model 2, with MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0007571498951329 % 


In [26]:
# Scenario 3  PRESENT DAY just for comparison to base case **

""" CHANGES: 
LAND_USE ARC_GRID ../input/Land_use_Soils_data/lu_grid.asc   to
LAND_USE ARC_GRID ../input/Land_use_Soils_data_Scen3/lu_grid.asc

Be sure to use: 
Direct_infiltration_input_folder = "Direct_infiltration_Scen_3"   """

Out_fold = os.path.join("..", "4_scn3_noclimate_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)
    
Control_File_Name = '4_scn3_noclimate_controlFile.ctl'
    
# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)

# RUN the model
Summary_frame_1 = run_model(Control_File_Name, Out_fold, x_dim, y_dim, Direct_infiltration_input_folder = "Direct_infiltration_Scen_3")

# delete the NetCDF files to clear up space (the data is already post processed into .asc's)
files_in_directory = os.listdir(Out_fold)
nc_files = [file for file in files_in_directory if file.endswith(".nc")]
for file in nc_files:
    path_to_file = os.path.join(Out_fold, file)
    os.remove(path_to_file)

4_scn3_noclimate_controlFile.ctl
Run model 1, no MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0092252500891026 % 
MFR total in MGD is 7.161306119793166
Caclculating MFR
Run model 2, with MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0091042641826515 % 


In [27]:
# BASE CASE FUTURE PRECIP LOW RESOLUTION

Out_fold = os.path.join("..", "5_FutureClimate_BASE_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)
    
    
""" CHANGES: 

PRECIPITATION ARC_GRID ../input/Gridded_rain/prism_ppt_tutuila_30yr_normal_%0m.asc    to 
PRECIPITATION ARC_GRID ../input/Gridded_rain_RCP_Present/cchanged_ppt_present_month_%m.asc

"""
    
Control_File_Name = '5_FutureClimate_BASE_controlFile.ctl'
    
# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)

# RUN the model
Summary_frame_1 = run_model(Control_File_Name, Out_fold, x_dim, y_dim, Direct_infiltration_input_folder = "Direct_infiltration")

# delete the NetCDF files to clear up space (the data is already post processed into .asc's)
files_in_directory = os.listdir(Out_fold)
nc_files = [file for file in files_in_directory if file.endswith(".nc")]
for file in nc_files:
    path_to_file = os.path.join(Out_fold, file)
    os.remove(path_to_file)

5_FutureClimate_BASE_controlFile.ctl
Run model 1, no MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0009419510523236 % 
MFR total in MGD is 6.612472956751569
Caclculating MFR
Run model 2, with MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.000930189982976 % 


In [28]:
# RCP4.5 FUTURE no LU change

Out_fold = os.path.join("..", "6_FutureClimate_RCP45_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)
    

""" CHANGES from CC base: 

PRECIPITATION ARC_GRID ../input/Gridded_rain_RCP_Present/cchanged_ppt_present_month_%m.asc   to 
PRECIPITATION ARC_GRID ../input/Gridded_rain_RCP_RCP4p5/cchanged_ppt_rcp45_month_%m.asc 

"""
    
Control_File_Name = '6_FutureClimate_RCP45_controlFile.ctl'


# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)

# RUN the model
Summary_frame_1 = run_model(Control_File_Name, Out_fold, x_dim, y_dim, Direct_infiltration_input_folder = "Direct_infiltration")
 
    # delete the NetCDF files to clear up space (the data is already post processed into .asc's)
files_in_directory = os.listdir(Out_fold)
nc_files = [file for file in files_in_directory if file.endswith(".nc")]
for file in nc_files:
    path_to_file = os.path.join(Out_fold, file)
    os.remove(path_to_file)

6_FutureClimate_RCP45_controlFile.ctl
Run model 1, no MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0050385449692238 % 
MFR total in MGD is 9.80401860653593
Caclculating MFR
Run model 2, with MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0049593626431341 % 


In [29]:
# RCP8.5 FUTURE no LU change

Out_fold = os.path.join("..", "7_FutureClimate_RCP85_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)
    
""" CHANGES from CC base: 

PRECIPITATION ARC_GRID ../input/Gridded_rain_RCP_Present/cchanged_ppt_present_month_%m.asc   to 
PRECIPITATION ARC_GRID ../input/Gridded_rain_RCP_RCP4p5/cchanged_ppt_rcp85_month_%m.asc 

"""
    
Control_File_Name = '7_FutureClimate_RCP85_controlFile.ctl'


# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)

# RUN the model
Summary_frame_1 = run_model(Control_File_Name, Out_fold, x_dim, y_dim, Direct_infiltration_input_folder = "Direct_infiltration")
 
    # delete the NetCDF files to clear up space (the data is already post processed into .asc's)
files_in_directory = os.listdir(Out_fold)
nc_files = [file for file in files_in_directory if file.endswith(".nc")]
for file in nc_files:
    path_to_file = os.path.join(Out_fold, file)
    os.remove(path_to_file)

7_FutureClimate_RCP85_controlFile.ctl
Run model 1, no MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.004146626674813 % 
MFR total in MGD is 8.732677860859337
Caclculating MFR
Run model 2, with MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0040849684535282 % 


In [30]:
# Present FUTURE PRECIP Scn1 LU  **

Out_fold = os.path.join("..", "8_Future_Present_Scn1_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)
    
""" CHANGES: 

PRECIPITATION ARC_GRID ../input/Gridded_rain/prism_ppt_tutuila_30yr_normal_%0m.asc    to 
PRECIPITATION ARC_GRID ../input/Gridded_rain_RCP_Present/cchanged_ppt_present_month_%m.asc

LAND_USE ARC_GRID ../input/Land_use_Soils_data/lu_grid.asc   to
LAND_USE ARC_GRID ../input/Land_use_Soils_data_Scen1/lu_grid.asc

Be sure to use: 
Direct_infiltration_input_folder = "Direct_infiltration_Scen_1"

"""   

Control_File_Name = '8_Future_Present_Scn1_controlFile.ctl'


# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)

# RUN the model
Summary_frame_1 = run_model(Control_File_Name, Out_fold, x_dim, y_dim, Direct_infiltration_input_folder = "Direct_infiltration_Scen_1")

# delete the NetCDF files to clear up space (the data is already post processed into .asc's)
files_in_directory = os.listdir(Out_fold)
nc_files = [file for file in files_in_directory if file.endswith(".nc")]
for file in nc_files:
    path_to_file = os.path.join(Out_fold, file)
    os.remove(path_to_file)

8_Future_Present_Scn1_controlFile.ctl
Run model 1, no MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0081203725956853 % 
MFR total in MGD is 6.840115089493725
Caclculating MFR
Run model 2, with MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0080160904123758 % 


In [31]:
# Present FUTURE PRECIP Scn2 LU  **

Out_fold = os.path.join("..", "9_Future_Present_Scn2_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)
    
    
""" CHANGES: 

PRECIPITATION ARC_GRID ../input/Gridded_rain/prism_ppt_tutuila_30yr_normal_%0m.asc    to 
PRECIPITATION ARC_GRID ../input/Gridded_rain_RCP_Present/cchanged_ppt_present_month_%m.asc

LAND_USE ARC_GRID ../input/Land_use_Soils_data/lu_grid.asc   to
LAND_USE ARC_GRID ../input/Land_use_Soils_data_Scen2/lu_grid.asc

Be sure to use: 
Direct_infiltration_input_folder = "Direct_infiltration_Scen_2"

"""   

Control_File_Name = '9_Future_Present_Scn2_controlFile.ctl'


# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)

# RUN the model
Summary_frame_1 = run_model(Control_File_Name, Out_fold, x_dim, y_dim, Direct_infiltration_input_folder = "Direct_infiltration_Scen_2")

# delete the NetCDF files to clear up space (the data is already post processed into .asc's)
files_in_directory = os.listdir(Out_fold)
nc_files = [file for file in files_in_directory if file.endswith(".nc")]
for file in nc_files:
    path_to_file = os.path.join(Out_fold, file)
    os.remove(path_to_file)


9_Future_Present_Scn2_controlFile.ctl
Run model 1, no MFR
WATER BALANCE ratio: outs over ins water budget balanece =  0.9979950948018135 % 
MFR total in MGD is 6.578975500808083
Caclculating MFR
Run model 2, with MFR
WATER BALANCE ratio: outs over ins water budget balanece =  0.9980200468099216 % 


In [32]:
# Present FUTURE PRECIP Scn3 LU  **

Out_fold = os.path.join("..", "10_Future_Present_Scn3_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)

    
        
""" CHANGES: 

PRECIPITATION ARC_GRID ../input/Gridded_rain/prism_ppt_tutuila_30yr_normal_%0m.asc    to 
PRECIPITATION ARC_GRID ../input/Gridded_rain_RCP_Present/cchanged_ppt_present_month_%m.asc

LAND_USE ARC_GRID ../input/Land_use_Soils_data/lu_grid.asc   to
LAND_USE ARC_GRID ../input/Land_use_Soils_data_Scen3/lu_grid.asc

Be sure to use: 
Direct_infiltration_input_folder = "Direct_infiltration_Scen_3"

"""   

Control_File_Name = '10_Future_Present_Scn3_controlFile.ctl'


# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)

# RUN the model
Summary_frame_1 = run_model(Control_File_Name, Out_fold, x_dim, y_dim, Direct_infiltration_input_folder = "Direct_infiltration_Scen_3")

# delete the NetCDF files to clear up space (the data is already post processed into .asc's)
files_in_directory = os.listdir(Out_fold)
nc_files = [file for file in files_in_directory if file.endswith(".nc")]
for file in nc_files:
    path_to_file = os.path.join(Out_fold, file)
    os.remove(path_to_file)

10_Future_Present_Scn3_controlFile.ctl
Run model 1, no MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.006589733000049 % 
MFR total in MGD is 6.947476426538538
Caclculating MFR
Run model 2, with MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.006503646641641 % 


In [33]:
# RCP4.5 FUTURE PRECIP Scn1 LU

Out_fold = os.path.join("..", "11_Future_RCP4.5_Scn1_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)
    
    
""" CHANGES: 

PRECIPITATION ARC_GRID ../input/Gridded_rain/prism_ppt_tutuila_30yr_normal_%0m.asc    to 
PRECIPITATION ARC_GRID ../input/Gridded_rain_RCP_RCP4p5/CChanged_ppt_rcp45_month_%m.asc

LAND_USE ARC_GRID ../input/Land_use_Soils_data/lu_grid.asc   to
LAND_USE ARC_GRID ../input/Land_use_Soils_data_Scen1/lu_grid.asc

Be sure to use: 
Direct_infiltration_input_folder = "Direct_infiltration_Scen_1"

"""   

Control_File_Name = '11_Future_RCP4.5_Scn1_controlFile.ctl'


# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)

# RUN the model
Summary_frame_1 = run_model(Control_File_Name, Out_fold, x_dim, y_dim, Direct_infiltration_input_folder = "Direct_infiltration_Scen_1")

# delete the NetCDF files to clear up space (the data is already post processed into .asc's)
files_in_directory = os.listdir(Out_fold)
nc_files = [file for file in files_in_directory if file.endswith(".nc")]
for file in nc_files:
    path_to_file = os.path.join(Out_fold, file)
    os.remove(path_to_file)

11_Future_RCP4.5_Scn1_controlFile.ctl
Run model 1, no MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0112968857750153 % 
MFR total in MGD is 10.106014067440936
Caclculating MFR
Run model 2, with MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0111147925363895 % 


In [34]:
# RCP4.5 FUTURE PRECIP Scn2 LU

Out_fold = os.path.join("..", "12_Future_RCP4.5_Scn2_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)
    
    
""" CHANGES: 

PRECIPITATION ARC_GRID ../input/Gridded_rain/prism_ppt_tutuila_30yr_normal_%0m.asc    to 
PRECIPITATION ARC_GRID ../input/Gridded_rain_RCP_RCP4p5/CChanged_ppt_rcp45_month_%m.asc

LAND_USE ARC_GRID ../input/Land_use_Soils_data/lu_grid.asc   to
LAND_USE ARC_GRID ../input/Land_use_Soils_data_Scen2/lu_grid.asc

Be sure to use: 
Direct_infiltration_input_folder = "Direct_infiltration_Scen_2"

"""   

Control_File_Name = '12_Future_RCP4.5_Scn2_controlFile.ctl'


# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)

# RUN the model
Summary_frame_1 = run_model(Control_File_Name, Out_fold, x_dim, y_dim, Direct_infiltration_input_folder = "Direct_infiltration_Scen_2")

# delete the NetCDF files to clear up space (the data is already post processed into .asc's)
files_in_directory = os.listdir(Out_fold)
nc_files = [file for file in files_in_directory if file.endswith(".nc")]
for file in nc_files:
    path_to_file = os.path.join(Out_fold, file)
    os.remove(path_to_file)

12_Future_RCP4.5_Scn2_controlFile.ctl
Run model 1, no MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0023513260066639 % 
MFR total in MGD is 9.755204967487463
Caclculating MFR
Run model 2, with MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0023145003807254 % 


In [35]:
# RCP4.5 FUTURE PRECIP Scn3 LU

Out_fold = os.path.join("..", "13_Future_RCP4.5_Scn3_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)
    
    
""" CHANGES: 

PRECIPITATION ARC_GRID ../input/Gridded_rain/prism_ppt_tutuila_30yr_normal_%0m.asc    to 
PRECIPITATION ARC_GRID ../input/Gridded_rain_RCP_RCP4p5/CChanged_ppt_rcp45_month_%m.asc

LAND_USE ARC_GRID ../input/Land_use_Soils_data/lu_grid.asc   to
LAND_USE ARC_GRID ../input/Land_use_Soils_data_Scen3/lu_grid.asc

Be sure to use: 
Direct_infiltration_input_folder = "Direct_infiltration_Scen_3"

"""   

Control_File_Name = '13_Future_RCP4.5_Scn3_controlFile.ctl'


# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)

# RUN the model
Summary_frame_1 = run_model(Control_File_Name, Out_fold, x_dim, y_dim, Direct_infiltration_input_folder = "Direct_infiltration_Scen_3")

# delete the NetCDF files to clear up space (the data is already post processed into .asc's)
files_in_directory = os.listdir(Out_fold)
nc_files = [file for file in files_in_directory if file.endswith(".nc")]
for file in nc_files:
    path_to_file = os.path.join(Out_fold, file)
    os.remove(path_to_file)

13_Future_RCP4.5_Scn3_controlFile.ctl
Run model 1, no MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0101816625097264 % 
MFR total in MGD is 10.24445498065541
Caclculating MFR
Run model 2, with MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0100150872243123 % 


In [36]:
# RCP8.5 FUTURE PRECIP Scn1 LU

Out_fold = os.path.join("..", "14_Future_RCP8.5_Scn1_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)
    
""" CHANGES: 

PRECIPITATION ARC_GRID ../input/Gridded_rain/prism_ppt_tutuila_30yr_normal_%0m.asc    to 
PRECIPITATION ARC_GRID ../input/Gridded_rain_RCP_RCP8p5/CChanged_ppt_rcp85_month_%m.asc

LAND_USE ARC_GRID ../input/Land_use_Soils_data/lu_grid.asc   to
LAND_USE ARC_GRID ../input/Land_use_Soils_data_Scen1/lu_grid.asc

Be sure to use: 
Direct_infiltration_input_folder = "Direct_infiltration_Scen_1"

"""   

Control_File_Name = '14_Future_RCP8.5_Scn1_controlFile.ctl'


# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)

# RUN the model
Summary_frame_1 = run_model(Control_File_Name, Out_fold, x_dim, y_dim, Direct_infiltration_input_folder = "Direct_infiltration_Scen_1")

# delete the NetCDF files to clear up space (the data is already post processed into .asc's)
files_in_directory = os.listdir(Out_fold)
nc_files = [file for file in files_in_directory if file.endswith(".nc")]
for file in nc_files:
    path_to_file = os.path.join(Out_fold, file)
    os.remove(path_to_file)

14_Future_RCP8.5_Scn1_controlFile.ctl
Run model 1, no MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0107164998456097 % 
MFR total in MGD is 9.003514705095037
Caclculating MFR
Run model 2, with MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0105530670575398 % 


In [37]:
# RCP8.5 FUTURE PRECIP Scn2 LU

Out_fold = os.path.join("..", "15_Future_RCP8.5_Scn2_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)
    
""" CHANGES: 

PRECIPITATION ARC_GRID ../input/Gridded_rain/prism_ppt_tutuila_30yr_normal_%0m.asc    to 
PRECIPITATION ARC_GRID ../input/Gridded_rain_RCP_RCP8p5/CChanged_ppt_rcp85_month_%m.asc

LAND_USE ARC_GRID ../input/Land_use_Soils_data/lu_grid.asc   to
LAND_USE ARC_GRID ../input/Land_use_Soils_data_Scen2/lu_grid.asc

Be sure to use: 
Direct_infiltration_input_folder = "Direct_infiltration_Scen_2"

"""   

Control_File_Name = '15_Future_RCP8.5_Scn2_controlFile.ctl'


# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)

# RUN the model
Summary_frame_1 = run_model(Control_File_Name, Out_fold, x_dim, y_dim, Direct_infiltration_input_folder = "Direct_infiltration_Scen_2")

# delete the NetCDF files to clear up space (the data is already post processed into .asc's)
files_in_directory = os.listdir(Out_fold)
nc_files = [file for file in files_in_directory if file.endswith(".nc")]
for file in nc_files:
    path_to_file = os.path.join(Out_fold, file)
    os.remove(path_to_file)

15_Future_RCP8.5_Scn2_controlFile.ctl
Run model 1, no MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0013772551201328 % 
MFR total in MGD is 8.698307204621594
Caclculating MFR
Run model 2, with MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0013568227860978 % 


In [38]:
# RCP8.5 FUTURE PRECIP Scn3 LU

Out_fold = os.path.join("..", "16_Future_RCP8.5_Scn3_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)
    
""" CHANGES: 

PRECIPITATION ARC_GRID ../input/Gridded_rain/prism_ppt_tutuila_30yr_normal_%0m.asc    to 
PRECIPITATION ARC_GRID ../input/Gridded_rain_RCP_RCP8p5/CChanged_ppt_rcp85_month_%m.asc

LAND_USE ARC_GRID ../input/Land_use_Soils_data/lu_grid.asc   to
LAND_USE ARC_GRID ../input/Land_use_Soils_data_Scen3/lu_grid.asc

Be sure to use: 
Direct_infiltration_input_folder = "Direct_infiltration_Scen_3"

"""   

Control_File_Name = '16_Future_RCP8.5_Scn3_controlFile.ctl'


# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)

# RUN the model
Summary_frame_1 = run_model(Control_File_Name, Out_fold, x_dim, y_dim, Direct_infiltration_input_folder = "Direct_infiltration_Scen_3")

# delete the NetCDF files to clear up space (the data is already post processed into .asc's)
files_in_directory = os.listdir(Out_fold)
nc_files = [file for file in files_in_directory if file.endswith(".nc")]
for file in nc_files:
    path_to_file = os.path.join(Out_fold, file)
    os.remove(path_to_file)

16_Future_RCP8.5_Scn3_controlFile.ctl
Run model 1, no MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0094530850459416 % 
MFR total in MGD is 9.132674262989534
Caclculating MFR
Run model 2, with MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0093066538193582 % 
